In [1]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

In [2]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [4]:
from bs4 import BeautifulSoup
import requests

# Data
### We obtain the postal codes of the Greater Vancouver Region from public sources along with the GPS coordinates
### We will then compare this information using Foursquare API to see the types and numbers of businesses in each of the neighbourhood.

# Methodology
#### 1. put the postal codes into dataframe 
#### 2. pull the foursquare data using the coordinates of 100 business within 500 meters
#### 3. compare the neighbourhoods and group those.
#### 4. cluster those neighbourhoods by similar business establishment and map the clusters. 
#### 5. investigate the clusters


In [5]:
df = pd.read_csv("vancouver coordinates 2.csv") 

In [6]:
df.head()

,V3A,Langley Township,(Langley City)-Langley Township,49.100002,-122.657128
0,V4A,Surrey,Southwest-Surrey,49.044655,-122.869163
1,V5A,Burnaby,(Government Road / Lake City / SFU / Burnaby M...,49.276301,-122.946971
2,V6A,Vancouver,(Strathcona / Chinatown / Downtown Eastside)- ...,49.279935,-123.090704
3,V7A,Richmond,South- Richmond,49.134223,-123.099148
4,V3B,Port Coquitlam,Central- Port Coquitlam,49.259167,-122.746993


In [7]:
df.columns=["Postal Code", "City", "Neighbourhood", "Latitude","Longitude"]

In [8]:
df.head()

,Postal Code,City,Neighbourhood,Latitude,Longitude
0,V4A,Surrey,Southwest-Surrey,49.044655,-122.869163
1,V5A,Burnaby,(Government Road / Lake City / SFU / Burnaby M...,49.276301,-122.946971
2,V6A,Vancouver,(Strathcona / Chinatown / Downtown Eastside)- ...,49.279935,-123.090704
3,V7A,Richmond,South- Richmond,49.134223,-123.099148
4,V3B,Port Coquitlam,Central- Port Coquitlam,49.259167,-122.746993


In [9]:
df.shape

(89, 5)

In [10]:
df.tail()

,Postal Code,City,Neighbourhood,Latitude,Longitude
84,V7Y,Vancouver,(Pacific Centre)-Vancouver,49.282728,-123.118463
85,V2Z,Langley Township,Southwest-Langley Township,49.065755,-122.582949
86,V3Z,Surrey,Lower East-Surrey,49.048837,-122.693318
87,V5Z,Vancouver,(East Fairview / South Cambie)-Vancouver,49.233483,-123.120701
88,V6Z,Vancouver,(SW Downtown)-Vancouver,49.275944,-123.131166


In [11]:
vancouver_data=df

In [12]:
CLIENT_ID = 'PNBZRK0AATNMVG3M1CVPRL2IYCPPD5VI52ROWJKW53OZDUXM' # Foursquare ID
CLIENT_SECRET = 'IDM0XD24USYGN05IGYR5VI35ELC1UCWUA11JM3MF3AUDLVR3' #  Foursquare Secret
VERSION = '20180604'

In [13]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=1000
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
vancouver_venues = getNearbyVenues(names=vancouver_data['Neighbourhood'],
                                   latitudes=vancouver_data['Latitude'],
                                   longitudes=vancouver_data['Longitude']
                                  )

Southwest-Surrey
(Government Road / Lake City / SFU / Burnaby Mountain)-Burnaby
(Strathcona / Chinatown / Downtown Eastside)- Vancouver
South- Richmond
Central- Port Coquitlam
(Parkcrest-Aubrey / Ardingley-Sprott)-Burnaby
(NE Downtown / Gastown / Harbour Centre / International Village / Victory Square / Yaletown)-Vancouver
(Sea Island / YVR)-Richmond
South-Port Coquitlam
Northeast-Delta
(Burnaby Heights / Willingdon Heights / West Central Valley)- Burnaby
(Waterfront / Coal Harbour / Canada Place)-Vancouver
Northwest-Richmond
East-Delta
(Lakeview-Mayfield / Richmond Park / Kingsway-Beresford)-Burnaby
(SE West End / Davie Village)-Vancouver
Southwest-Richmond
East Central-Delta
(Cascade-Schou / Douglas-Gilpin)-Burnaby
(NW West End / Stanley Park)-Vancouver
Outer East-North Vancouver (district municipality)
Port Moody
(Maywood / Marlborough / Oakalla / Windsor)-Burnaby
(West Fairview / Granville Island / NE Shaughnessy)-Vancouver
Inner East-North Vancouver (district municipality)
(Suncre

In [15]:
vancouver_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Southwest-Surrey,49.044655,-122.869163,Crescent Park,49.049943,-122.862741,Park
1,Southwest-Surrey,49.044655,-122.869163,Fetch 'n Frolic Pet Services,49.038265,-122.875547,Hot Dog Joint
2,Southwest-Surrey,49.044655,-122.869163,Esquires Coffee House,49.053296,-122.866983,Café
3,Southwest-Surrey,49.044655,-122.869163,Fieldstone Artisan Breads,49.053410,-122.867274,Bakery
4,(Government Road / Lake City / SFU / Burnaby M...,49.276301,-122.946971,Hop & Vine Tap House,49.269634,-122.946533,Burger Joint


In [16]:
vancouver_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
(Bentall Centre)-Vancouver,100,100,100,100,100,100
(Burnaby Heights / Willingdon Heights / West Central Valley)- Burnaby,59,59,59,59,59,59
(Cascade-Schou / Douglas-Gilpin)-Burnaby,21,21,21,21,21,21
(Central Kitsilano / Greektown)-Vancouver,88,88,88,88,88,88
(East Big Bend / Stride Avenue / Edmonds / Cariboo-Armstrong)-Burnaby,39,39,39,39,39,39
...,...,...,...,...,...,...
Southwest-Surrey,4,4,4,4,4,4
Upper East-Surrey,41,41,41,41,41,41
Upper West-Surrey,44,44,44,44,44,44


In [17]:
# one hot encoding
vancouver_onehot = pd.get_dummies(vancouver_venues[['Venue Category']], prefix="", prefix_sep="")

In [18]:
vancouver_onehot['Neighbourhood'] = vancouver_venues['Neighbourhood'] 

In [19]:
fixed_columns = [vancouver_onehot.columns[-1]] + list(vancouver_onehot.columns[:-1])
vancouver_onehot = vancouver_onehot[fixed_columns]

vancouver_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,...,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio,Zoo
0,Southwest-Surrey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Southwest-Surrey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Southwest-Surrey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Southwest-Surrey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,(Government Road / Lake City / SFU / Burnaby M...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
vancouver_onehot.shape

(3742, 303)

In [21]:
vancouver_grouped = vancouver_onehot.groupby('Neighbourhood').mean().reset_index()
vancouver_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,...,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio,Zoo
0,(Bentall Centre)-Vancouver,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,...,0.010000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.010000,0.0
1,(Burnaby Heights / Willingdon Heights / West C...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.016949,0.0,...,0.016949,0.016949,0.0,0.0,0.0,0.016949,0.0,0.000000,0.000000,0.0
2,(Cascade-Schou / Douglas-Gilpin)-Burnaby,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
3,(Central Kitsilano / Greektown)-Vancouver,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.011364,0.000000,0.0,0.0,0.0,0.011364,0.0,0.011364,0.011364,0.0
4,(East Big Bend / Stride Avenue / Edmonds / Car...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.025641,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,Southwest-Surrey,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
85,Upper East-Surrey,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
86,Upper West-Surrey,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022727,0.0,...,0.090909,0.022727,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
87,West-Maple Ridge,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0


In [22]:
vancouver_grouped.shape

(89, 303)

In [23]:
num_top_venues = 5

for hood in vancouver_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = vancouver_grouped[vancouver_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----(Bentall Centre)-Vancouver----
                 venue  freq
0                Hotel  0.09
1         Dessert Shop  0.05
2          Coffee Shop  0.05
3  Japanese Restaurant  0.04
4       Sandwich Place  0.04


----(Burnaby Heights / Willingdon Heights / West Central Valley)- Burnaby----
                  venue  freq
0           Coffee Shop  0.10
1      Sushi Restaurant  0.05
2              Pharmacy  0.05
3            Restaurant  0.03
4  Fast Food Restaurant  0.03


----(Cascade-Schou / Douglas-Gilpin)-Burnaby----
                 venue  freq
0             Bus Stop  0.10
1          Coffee Shop  0.10
2                 Park  0.10
3    Indian Restaurant  0.10
4  Arts & Crafts Store  0.05


----(Central Kitsilano / Greektown)-Vancouver----
                       venue  freq
0                Coffee Shop  0.12
1        Japanese Restaurant  0.05
2           Sushi Restaurant  0.03
3  Middle Eastern Restaurant  0.03
4                       Park  0.03


----(East Big Bend / Stride Avenue / Edmon

### put into pandas dataframe

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = vancouver_grouped['Neighbourhood']

for ind in np.arange(vancouver_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(vancouver_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,(Bentall Centre)-Vancouver,Hotel,Dessert Shop,Coffee Shop,Restaurant,Sandwich Place,Japanese Restaurant,Café,Concert Hall,Bookstore,Cosmetics Shop
1,(Burnaby Heights / Willingdon Heights / West C...,Coffee Shop,Sushi Restaurant,Pharmacy,Fast Food Restaurant,Park,Burger Joint,Restaurant,Light Rail Station,Sporting Goods Shop,Pizza Place
2,(Cascade-Schou / Douglas-Gilpin)-Burnaby,Indian Restaurant,Bus Stop,Park,Coffee Shop,Gym,Gym / Fitness Center,Cafeteria,American Restaurant,Sandwich Place,Convenience Store
3,(Central Kitsilano / Greektown)-Vancouver,Coffee Shop,Japanese Restaurant,Café,Sushi Restaurant,Park,Tea Room,Bakery,Vegetarian / Vegan Restaurant,Middle Eastern Restaurant,Ice Cream Shop
4,(East Big Bend / Stride Avenue / Edmonds / Car...,Sandwich Place,Café,Chinese Restaurant,Sushi Restaurant,Convenience Store,Grocery Store,Thai Restaurant,Bank,Sporting Goods Shop,Breakfast Spot


## cluster the neighbourhoods into 5 clusters

In [26]:
# set number of clusters
kclusters = 5

vancouver_grouped_clustering = vancouver_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(vancouver_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 2, 0, 0, 0, 0, 0, 0], dtype=int32)

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)



In [28]:
neighborhoods_venues_sorted # check the data

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,(Bentall Centre)-Vancouver,Hotel,Dessert Shop,Coffee Shop,Restaurant,Sandwich Place,Japanese Restaurant,Café,Concert Hall,Bookstore,Cosmetics Shop
1,2,(Burnaby Heights / Willingdon Heights / West C...,Coffee Shop,Sushi Restaurant,Pharmacy,Fast Food Restaurant,Park,Burger Joint,Restaurant,Light Rail Station,Sporting Goods Shop,Pizza Place
2,0,(Cascade-Schou / Douglas-Gilpin)-Burnaby,Indian Restaurant,Bus Stop,Park,Coffee Shop,Gym,Gym / Fitness Center,Cafeteria,American Restaurant,Sandwich Place,Convenience Store
3,2,(Central Kitsilano / Greektown)-Vancouver,Coffee Shop,Japanese Restaurant,Café,Sushi Restaurant,Park,Tea Room,Bakery,Vegetarian / Vegan Restaurant,Middle Eastern Restaurant,Ice Cream Shop
4,0,(East Big Bend / Stride Avenue / Edmonds / Car...,Sandwich Place,Café,Chinese Restaurant,Sushi Restaurant,Convenience Store,Grocery Store,Thai Restaurant,Bank,Sporting Goods Shop,Breakfast Spot
...,...,...,...,...,...,...,...,...,...,...,...,...
84,0,Southwest-Surrey,Bakery,Park,Hot Dog Joint,Café,Flea Market,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market
85,2,Upper East-Surrey,Coffee Shop,Salon / Barbershop,Pharmacy,Fast Food Restaurant,Gym / Fitness Center,Bistro,Market,Flea Market,Neighborhood,Salad Place
86,2,Upper West-Surrey,Coffee Shop,Pizza Place,Vietnamese Restaurant,Fast Food Restaurant,Pub,Bank,Electronics Store,Gas Station,Grocery Store,Pet Store
87,0,West-Maple Ridge,Fast Food Restaurant,Hotel,Liquor Store,Bakery,Gas Station,Park,Italian Restaurant,Restaurant,Track,Zoo


In [29]:
vancouver_data  #check data

,Postal Code,City,Neighbourhood,Latitude,Longitude
0,V4A,Surrey,Southwest-Surrey,49.044655,-122.869163
1,V5A,Burnaby,(Government Road / Lake City / SFU / Burnaby M...,49.276301,-122.946971
2,V6A,Vancouver,(Strathcona / Chinatown / Downtown Eastside)- ...,49.279935,-123.090704
3,V7A,Richmond,South- Richmond,49.134223,-123.099148
4,V3B,Port Coquitlam,Central- Port Coquitlam,49.259167,-122.746993
...,...,...,...,...,...
84,V7Y,Vancouver,(Pacific Centre)-Vancouver,49.282728,-123.118463
85,V2Z,Langley Township,Southwest-Langley Township,49.065755,-122.582949
86,V3Z,Surrey,Lower East-Surrey,49.048837,-122.693318
87,V5Z,Vancouver,(East Fairview / South Cambie)-Vancouver,49.233483,-123.120701


In [30]:

vancouver_merged = vancouver_data

# merge vancouver_grouped with vancouver_data to add latitude/longitude for each neighbuorhood
vancouver_merged = vancouver_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood', how='inner')

vancouver_merged.head() # check the last columns!

,Postal Code,City,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,V4A,Surrey,Southwest-Surrey,49.044655,-122.869163,0,Bakery,Park,Hot Dog Joint,Café,Flea Market,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market
1,V5A,Burnaby,(Government Road / Lake City / SFU / Burnaby M...,49.276301,-122.946971,0,Bubble Tea Shop,Bus Line,Grocery Store,Park,Bus Stop,Baseball Field,Pizza Place,Burger Joint,Forest,French Restaurant
2,V6A,Vancouver,(Strathcona / Chinatown / Downtown Eastside)- ...,49.279935,-123.090704,0,Café,Park,Sandwich Place,Bar,Coffee Shop,Restaurant,Pizza Place,Chinese Restaurant,Brewery,Beer Bar
3,V7A,Richmond,South- Richmond,49.134223,-123.099148,0,Shopping Mall,Sporting Goods Shop,Restaurant,Gym,Sushi Restaurant,Vietnamese Restaurant,Coffee Shop,Brewery,Breakfast Spot,Shanghai Restaurant
4,V3B,Port Coquitlam,Central- Port Coquitlam,49.259167,-122.746993,0,Fast Food Restaurant,Coffee Shop,Bank,Discount Store,Pet Store,Restaurant,Vietnamese Restaurant,Big Box Store,Theater,Greek Restaurant


In [31]:
vancouver_merged.shape

(89, 16)

In [32]:
import folium 

In [33]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [34]:
from geopy.geocoders import Nominatim 

In [35]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [36]:
address = 'Vancouver, CA'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vancouver are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vancouver are 49.2608724, -123.1139529.


In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(vancouver_merged['Latitude'], vancouver_merged['Longitude'], vancouver_merged['Neighbourhood'], vancouver_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine the clusters

### Cluster 1

In [38]:
vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 0, vancouver_merged.columns[[2] + list(range(5, vancouver_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Southwest-Surrey,0,Bakery,Park,Hot Dog Joint,Café,Flea Market,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market
1,(Government Road / Lake City / SFU / Burnaby M...,0,Bubble Tea Shop,Bus Line,Grocery Store,Park,Bus Stop,Baseball Field,Pizza Place,Burger Joint,Forest,French Restaurant
2,(Strathcona / Chinatown / Downtown Eastside)- ...,0,Café,Park,Sandwich Place,Bar,Coffee Shop,Restaurant,Pizza Place,Chinese Restaurant,Brewery,Beer Bar
3,South- Richmond,0,Shopping Mall,Sporting Goods Shop,Restaurant,Gym,Sushi Restaurant,Vietnamese Restaurant,Coffee Shop,Brewery,Breakfast Spot,Shanghai Restaurant
4,Central- Port Coquitlam,0,Fast Food Restaurant,Coffee Shop,Bank,Discount Store,Pet Store,Restaurant,Vietnamese Restaurant,Big Box Store,Theater,Greek Restaurant
5,(Parkcrest-Aubrey / Ardingley-Sprott)-Burnaby,0,Sushi Restaurant,Diner,Vietnamese Restaurant,Grocery Store,Chinese Restaurant,Shopping Mall,Coffee Shop,Gas Station,Italian Restaurant,Soccer Field
6,(NE Downtown / Gastown / Harbour Centre / Inte...,0,Hotel,Coffee Shop,Restaurant,Café,Sandwich Place,Taco Place,Breakfast Spot,Dessert Shop,Concert Hall,Clothing Store
9,Northeast-Delta,0,Indian Restaurant,Discount Store,Restaurant,Breakfast Spot,Electronics Store,Sushi Restaurant,Szechuan Restaurant,Chinese Restaurant,Sandwich Place,Filipino Restaurant
11,(Waterfront / Coal Harbour / Canada Place)-Van...,0,Hotel,Coffee Shop,Café,Sandwich Place,Dessert Shop,Vegetarian / Vegan Restaurant,Lounge,American Restaurant,Cosmetics Shop,Breakfast Spot
12,Northwest-Richmond,0,Japanese Restaurant,Frozen Yogurt Shop,Coffee Shop,Bus Stop,German Restaurant,Sushi Restaurant,Shopping Mall,Liquor Store,Farmers Market,Thai Restaurant


### Cluster 2

In [39]:
vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 1, vancouver_merged.columns[[2] + list(range(5, vancouver_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
71,East-Langley Township,1,Exhibit,Zoo,Intersection,Flower Shop,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market


### Cluser 3

In [40]:
vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 2, vancouver_merged.columns[[2] + list(range(5, vancouver_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,(Sea Island / YVR)-Richmond,2,Coffee Shop,Airport Service,Airport Lounge,Rental Car Location,Sandwich Place,Chocolate Shop,Hotel,Bakery,Bank,Public Art
8,South-Port Coquitlam,2,Coffee Shop,Grocery Store,Pharmacy,Convenience Store,Pizza Place,Pub,Ice Cream Shop,Bakery,Sandwich Place,Sushi Restaurant
10,(Burnaby Heights / Willingdon Heights / West C...,2,Coffee Shop,Sushi Restaurant,Pharmacy,Fast Food Restaurant,Park,Burger Joint,Restaurant,Light Rail Station,Sporting Goods Shop,Pizza Place
20,Outer East-North Vancouver (district municipal...,2,Bank,Restaurant,Food & Drink Shop,Grocery Store,Park,Coffee Shop,Japanese Restaurant,Sporting Goods Shop,Pub,Dog Run
21,Port Moody,2,Coffee Shop,Sushi Restaurant,Gym,Burger Joint,Pub,Grocery Store,Bus Stop,Bank,Bakery,Café
27,East Central-North Vancouver (district municip...,2,Coffee Shop,Pub,Sporting Goods Shop,Fast Food Restaurant,Arts & Crafts Store,Grocery Store,Gastropub,Market,Gas Station,Breakfast Spot
30,(Central Kitsilano / Greektown)-Vancouver,2,Coffee Shop,Japanese Restaurant,Café,Sushi Restaurant,Park,Tea Room,Bakery,Vegetarian / Vegan Restaurant,Middle Eastern Restaurant,Ice Cream Shop
31,North Central-North Vancouver (district munici...,2,Coffee Shop,Fast Food Restaurant,Grocery Store,Bank,Shopping Mall,Convenience Store,Japanese Restaurant,Sandwich Place,Gastropub,Breakfast Spot
33,Southeast-Delta,2,Coffee Shop,American Restaurant,Grocery Store,Bank,Italian Restaurant,Burger Joint,Shopping Mall,Gas Station,Farmers Market,Sushi Restaurant
34,(North Grandview)Vancouver,2,Brewery,Coffee Shop,Café,Pizza Place,Sushi Restaurant,Ice Cream Shop,Breakfast Spot,Japanese Restaurant,Italian Restaurant,Bakery


### Cluster 4

In [41]:
vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 3, vancouver_merged.columns[[2] + list(range(5, vancouver_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Northwest-Delta,3,Winery,Coffee Shop,Zoo,Flea Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market


### Cluster 5

In [42]:
vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 4, vancouver_merged.columns[[2] + list(range(5, vancouver_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Northeast-Surrey,4,Flower Shop,Trail,Flea Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Zoo
85,Southwest-Langley Township,4,Flower Shop,Construction & Landscaping,Zoo,Food,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market
